In [ ]:
pip install freesurfer-stats

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from freesurfer_stats import CorticalParcellationStats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
datadir = '/home/jovyan/shared/ds003097/'

In [3]:
def get_all_regions(whichdata=None):
    all_sub_dfs = []
    for sub in tqdm(range(1, 101), desc='Subjects'):
        lh_fname = f'/home/jovyan/shared/ds003097/derivatives/freesurfer/sub-{sub:04}/stats/lh.aparc.a2009s.stats'
        rh_fname = f'/home/jovyan/shared/ds003097/derivatives/freesurfer/sub-{sub:04}/stats/rh.aparc.a2009s.stats'
        try:
            lh_stats = CorticalParcellationStats.read(lh_fname)
            rh_stats = CorticalParcellationStats.read(rh_fname)
        except FileNotFoundError:
            continue
        if whichdata == 'SurfArea':
            lh_stats_df = lh_stats.structural_measurements[['surface_area_mm^2']].copy()
            rh_stats_df = rh_stats.structural_measurements[['surface_area_mm^2']].copy()
        elif whichdata == 'GrayVol':
            lh_stats_df = lh_stats.structural_measurements[['gray_matter_volume_mm^3']].copy()
            rh_stats_df = rh_stats.structural_measurements[['gray_matter_volume_mm^3']].copy()
        elif whichdata == 'AvgThick':
            lh_stats_df = lh_stats.structural_measurements[['average_thickness_mm']].copy()
            rh_stats_df = rh_stats.structural_measurements[['average_thickness_mm']].copy()
        elif whichdata == None:
            lh_stats_df = lh_stats.structural_measurements[['surface_area_mm^2', 'gray_matter_volume_mm^3', 
                                                            'average_thickness_mm']].copy()
            rh_stats_df = rh_stats.structural_measurements[['surface_area_mm^2', 'gray_matter_volume_mm^3', 
                                                            'average_thickness_mm']].copy()
        
        stats_df = pd.concat([lh_stats_df, rh_stats_df])
        stats_df = stats_df.transpose()
        stats_df['sub_id'] = f'sub-{sub:04}'
        all_sub_dfs.append(stats_df)

    all_sub_df = pd.concat(all_sub_dfs)
    return all_sub_df, lh_stats_df, rh_stats_df


In [19]:
[all_sub_anat, lh_stats_df, rh_stats_df] = get_all_regions('GrayVol')
all_sub_anat

Subjects:   0%|          | 0/100 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,sub_id
gray_matter_volume_mm^3,2159,3141,2874,3126,1599,4958,2483,1950,1559,550,...,1738,3290,2100,1695,328,2216,2171,11704,329,sub-0001
gray_matter_volume_mm^3,2899,2645,2493,2902,1902,4679,2378,2411,1579,708,...,1071,2975,2590,2213,593,1540,2129,12786,612,sub-0002
gray_matter_volume_mm^3,2644,2353,2535,2731,1478,4446,2125,2673,1258,526,...,799,2905,2431,2027,357,1572,1652,8333,336,sub-0003
gray_matter_volume_mm^3,2137,2763,3341,3658,1804,4846,2676,3840,1479,616,...,1643,3407,2836,2241,619,1689,1881,9785,454,sub-0004
gray_matter_volume_mm^3,2568,3549,2807,4297,2328,6495,2762,2615,1915,819,...,1866,3468,2978,3440,560,3181,2017,10969,388,sub-0005
gray_matter_volume_mm^3,2178,2724,2460,2071,1338,3601,2053,2581,1173,583,...,942,2741,1862,1784,373,1971,1363,9021,359,sub-0006
gray_matter_volume_mm^3,2293,3733,2728,3399,2101,5161,2971,2233,1637,872,...,1565,3300,3005,2189,642,2439,1632,12927,646,sub-0007
gray_matter_volume_mm^3,2133,3068,2506,3453,2030,4813,1800,2369,1972,580,...,1140,2842,2558,1528,543,2465,2523,10251,341,sub-0008
gray_matter_volume_mm^3,2499,3567,2725,4740,2528,6154,3488,3491,1647,614,...,1403,4508,2644,2045,853,2859,2638,9318,453,sub-0009


In [20]:
# get X (sub x anatomical feature from all regions)
X = all_sub_anat.iloc[:,0:74].to_numpy()
np.shape(X)

(9, 74)

In [21]:
# house keeping
nX = np.shape(X)[0]
pTrain = .8
ntrain = round(nX * pTrain)

In [22]:
# read psychometric data
df = pd.read_csv(f'{datadir}participants.tsv', sep="\t") 
df

,participant_id,age,sex,handedness,BMI,education_level,background_SES,IST_fluid,IST_memory,IST_crystallised,...,sexual_attraction_M,sexual_attraction_F,gender_identity_M,gender_identity_F,religious_upbringing,religious_now,religious_importance,DWI_TR_run1,DWI_TR_run2,DWI_TR_run3
0,sub-0001,22.00,female,right,23,medium,2.0,77.0,49.0,33.0,...,7.0,1.0,1.0,7.0,no,yes,2.0,6.312,6.312,6.312
1,sub-0002,21.75,female,right,20,medium,5.5,97.0,63.0,39.0,...,7.0,1.0,2.0,7.0,no,no,NaN,NaN,6.311,6.311
2,sub-0003,25.25,female,right,31,high,3.0,122.0,67.0,38.0,...,6.0,3.0,1.0,6.0,no,no,NaN,6.312,6.312,6.312
3,sub-0004,22.50,female,right,20,high,5.0,149.0,69.0,52.0,...,6.0,2.0,1.0,7.0,yes,no,NaN,6.311,6.311,6.311
4,sub-0005,22.25,male,right,23,high,4.5,112.0,57.0,43.0,...,1.0,7.0,6.0,1.0,no,no,NaN,6.311,6.311,6.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,sub-0924,22.25,male,right,21,medium,3.0,136.0,56.0,54.0,...,2.0,6.0,4.0,4.0,no,no,NaN,6.374,6.374,6.374
924,sub-0925,25.25,male,right,30,medium,4.0,64.0,37.0,49.0,...,1.0,7.0,7.0,1.0,no,no,NaN,6.311,6.311,6.311
925,sub-0926,20.75,male,right,22,high,2.0,84.0,44.0,33.0,...,NaN,NaN,NaN,NaN,yes,yes,5.0,6.312,6.312,6.312
926,sub-0927,24.25,female,right,35,medium,2.5,98.0,57.0,35.0,...,7.0,2.0,1.0,7.0,no,no,NaN,6.311,6.311,6.311


In [23]:
list(df.columns)

['participant_id',
 'age',
 'sex',
 'handedness',
 'BMI',
 'education_level',
 'background_SES',
 'IST_fluid',
 'IST_memory',
 'IST_crystallised',
 'IST_intelligence_total',
 'BAS_drive',
 'BAS_fun',
 'BAS_reward',
 'BIS',
 'NEO_N',
 'NEO_E',
 'NEO_O',
 'NEO_A',
 'NEO_C',
 'STAI_T',
 'sexual_attraction_M',
 'sexual_attraction_F',
 'gender_identity_M',
 'gender_identity_F',
 'religious_upbringing',
 'religious_now',
 'religious_importance',
 'DWI_TR_run1',
 'DWI_TR_run2',
 'DWI_TR_run3']

In [24]:
# get Y (IST intelligence total)
Y = df.iloc[0:nX,10].values
nY = np.shape(Y)[0]
Y

array([159., 199., 227., 270., 212.,  96., 172., 162., 160.])

In [25]:
nX == nY # sanity check

True

In [26]:
np.random.seed(0)

In [27]:
# randomly select test and train
idx = np.arange(nX)
np.random.shuffle(idx) 
idy = np.arange(nY)
np.random.shuffle(idy)

In [28]:
X_train = X[idx[0:ntrain],:]
X_test = X[idx[ntrain:],:]
Y_train = Y[idy[0:ntrain],]
Y_test = Y[idy[ntrain:]]

In [29]:
# model, train, and test
reg = linear_model.Ridge(alpha=1)
reg.fit(X_train, Y_train)
Y_pred = reg.predict(X_test)

In [30]:
Y_pred

array([205.39394754, 247.957039  ])

In [31]:
Y_test

array([227.,  96.])

In [32]:
print("Coefficients: \n", reg.coef_)
print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
print("Coefficient of determination: %.2f" % r2_score(Y_test, Y_pred))

Coefficients: 
 [-3.88664782e-03  1.92500987e-03 -1.03637575e-04  2.44713859e-03
  1.00389142e-03  2.35916824e-04 -4.45226647e-03 -1.19596300e-03
  3.06692245e-03 -5.13488066e-04 -1.27930794e-03 -3.24378003e-03
 -9.62732924e-04 -1.67493481e-03 -7.14757266e-03 -7.52819185e-03
 -1.80158467e-03 -8.77715604e-04  4.54213077e-03 -1.45459796e-03
  2.67850048e-03 -2.48128107e-03  6.13499803e-03 -1.35145187e-03
  2.38517376e-03  3.07649355e-04 -5.99660271e-03 -4.99121036e-04
 -9.37838412e-04  5.43029987e-03  1.21406647e-03  1.55617378e-03
 -2.53614306e-03 -1.59065861e-03 -2.24784333e-03  3.06701950e-03
  7.88604005e-03 -7.28285223e-03 -4.76968546e-04 -2.66944715e-03
  1.90034339e-03  3.97083327e-04 -5.00654732e-05  1.08138439e-03
  2.08902721e-03  3.02646097e-04 -1.25926202e-03 -2.32309221e-03
 -1.84336294e-03  2.66219164e-04 -1.38265244e-03  2.18327167e-03
 -3.35780831e-03 -2.29021230e-03 -5.23723419e-04  2.95564899e-03
  3.79454874e-03 -2.39024496e-03  1.58329477e-04 -2.68755191e-04
 -5.25576